<a href="https://colab.research.google.com/github/Teasotea/textSummarization/blob/main/Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries

In [65]:
!pip install transformers sentencepiece
!pip install sentence_transformers

In [66]:
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import time
from sklearn.metrics.pairwise import cosine_similarity
pd.set_option('display.max_colwidth', None)

In [67]:
# mount gdrive to save models
from pathlib import Path
from google.colab import drive

drive.mount('/content/gdrive')

GDRIVE = '/content/gdrive/MyDrive/transformers'
WORKDIR = GDRIVE + '/sent_ranking/'
Path(WORKDIR).mkdir(parents=True, exist_ok=True)

%cd  '/content/gdrive/MyDrive/text_summ/transformers'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/MyDrive/text_summ/transformers


# Load Data

In [68]:
conversation_df = pd.read_csv('https://raw.githubusercontent.com/Teasotea/textSummarization/main/data/session.csv', delimiter = "\t", header=None)
conversation_df = conversation_df.set_axis(['text'], axis=1, inplace=False)
conversation_df.style.set_properties(**{'text-align': 'left'})

,text
0,"Психолог: Добрый день, меня зовут Ирина. Ольга, у нас с вами на сегодня назначена консультация. Надеюсь, что наш разговор будет приятным и полезным. Расскажите с каким запросом вы пришли на нашу первую встречу?"
1,"Ольга: Добрый день, это моя первая консультация в жизни. Я не особо понимаю что именно нужно говорить. Но у меня накопилось достаточно много всего и мне просто хотелось поговорить о том, что сейчас происходит с моей жизнью, ну много чего, конечно, как и у всех, все таки война, переезды, куча всего короче. Просто хотелось чтобы кто-то услышал, помог мне немного разобраться в себе."
2,"Психолог: Ольга, действительно сейчас многие проживают события к которым сложно быть готовым. Давайте начнем понемногу разбираться. Расскажите о себе. Где вы сейчас находитесь, в каких условиях живете, какие события происходят вокруг вас?"
3,"Ольга: Сейчас я чувствую себя относительно в безопасности. Мы переехали из Киева на западную Украину. Здесь можно сказать, что тихо, но я не особо понимаю что происходит. Мне словно приходится начинать все с нуля. Деньги немного есть, ээээ ну там на пару месяцев хватит, но нужно искать работу. Не пойму нужно учится на новую специальность или попытаться найти работу менеджером, как раньше? Хотя кому эти менеджеры нужны, ничего толком не работает, наверно повезло людям которые сохранили работу, прям как-то радостно за них. Потерять финансовую стабильность мне реально тяжело, ждать что что-то наладится, но ничего не наладится же. Ну даже если наладится то не скоро, рада что могла себе обеспечить эту финансовую подушку. Я реально уже думала что можно и начать инвестировать но блин, реально никогда не знаешь, теперь думаю что можно было и пол года назад параллельно с подушкой все делать, но так иногда радует что хоть стабильность в деньгах есть, я все таки смогла научиться управлять немного своими финансами. Ну и новую работу искать, ну и что я тут найду? Наверно скорее всего нужно будет учить английский, вряд ли украинские компании сейчас смогут дать хорошую оплату, надо искать что-то онлайн, стабильное. Хотя какое оно стабильное, надо открывать свое дело в интернете, может делать магазин с доставкой? Да я так и не смогла этот английский выучить сколько уже ходила на курсы. Короче искать работу это мне вообще как настоящее испытание, ну и не думаю что без английского я кому-то нужна. Пока живем в бесплатном жилье, но тоже непонятно надолго это, вообще нужно ли возвращаться, в Киеве хотя бы есть дом, родные стены, ну и удобства все под рукой, но я боюсь ходить по улице вижу везде эти новости про растяжки, бомбы в молоке. Но и помощи от волонтеров больше есть и помощь с едой и вообще дома как то проще, здесь никого не знаю, но и знакомые если честно все уже разъехались и реально жизнь как с нуля. С мужем тоже сложно, постоянно приходится принимать решения, а он как будто оттягивает, но я же тоже не могу все время все делать. Короче мне реально не понятно как вообще быть, я боюсь что-то снова начинать, потому что не понятно на долго или скоро закончится."
4,"Психолог: Давайте выделим вопросы, которые я услышала и продолжим работу. Сейчас у вас нет основной работы для вас большой стресс искать новую, еще вы хотите вернуться в Киев, но не можете решить для себя стоит ли это сейчас делать, также вам хочется получить поддержку от мужа, но вы не можете получить той помощи на которую рассчитываете. Но самым большим испытанием для вас является то, что необходимо начать все с нуля. верно?"
5,"Ольга: Да, но даже не сам факт начала. Понимаете я уже несколько раз все теряла и начинала ""жизнь с этого нуля"". Поэтому мне кажется тут у меня что-то вроде иммунитета. Страх не с самой необходимости, я просто реально понимаю что если я еще раз все потеряю мне может просто не хотеться что-то делать снова. Именно когда живешь в период от нуля до ""опять все хочется"" тут самое страшное. Ощущение, что теряешь время, которого и так постоянно мало, и параллельно же ничего не улучшается, тебе еще наваливается сверху, и

In [69]:
conversation_df['interlocutor'] = conversation_df.apply(lambda x: 'cli' if x.text.lower().startswith('о') else 'psy', axis=1)
conversation_df['text'] = conversation_df['text'].apply(lambda x: x.strip().split(' ', 1)[1] if x.strip().lower().startswith('психолог') | x.strip().lower().startswith('о') else x.strip())
psy_df = conversation_df[conversation_df['interlocutor'] == 'psy']
cli_df = conversation_df[conversation_df['interlocutor'] == 'cli']
conversation_df.style.set_properties(**{'text-align': 'left'})

,text,interlocutor
0,"Добрый день, меня зовут Ирина. Ольга, у нас с вами на сегодня назначена консультация. Надеюсь, что наш разговор будет приятным и полезным. Расскажите с каким запросом вы пришли на нашу первую встречу?",psy
1,"Добрый день, это моя первая консультация в жизни. Я не особо понимаю что именно нужно говорить. Но у меня накопилось достаточно много всего и мне просто хотелось поговорить о том, что сейчас происходит с моей жизнью, ну много чего, конечно, как и у всех, все таки война, переезды, куча всего короче. Просто хотелось чтобы кто-то услышал, помог мне немного разобраться в себе.",cli
2,"Ольга, действительно сейчас многие проживают события к которым сложно быть готовым. Давайте начнем понемногу разбираться. Расскажите о себе. Где вы сейчас находитесь, в каких условиях живете, какие события происходят вокруг вас?",psy
3,"Сейчас я чувствую себя относительно в безопасности. Мы переехали из Киева на западную Украину. Здесь можно сказать, что тихо, но я не особо понимаю что происходит. Мне словно приходится начинать все с нуля. Деньги немного есть, ээээ ну там на пару месяцев хватит, но нужно искать работу. Не пойму нужно учится на новую специальность или попытаться найти работу менеджером, как раньше? Хотя кому эти менеджеры нужны, ничего толком не работает, наверно повезло людям которые сохранили работу, прям как-то радостно за них. Потерять финансовую стабильность мне реально тяжело, ждать что что-то наладится, но ничего не наладится же. Ну даже если наладится то не скоро, рада что могла себе обеспечить эту финансовую подушку. Я реально уже думала что можно и начать инвестировать но блин, реально никогда не знаешь, теперь думаю что можно было и пол года назад параллельно с подушкой все делать, но так иногда радует что хоть стабильность в деньгах есть, я все таки смогла научиться управлять немного своими финансами. Ну и новую работу искать, ну и что я тут найду? Наверно скорее всего нужно будет учить английский, вряд ли украинские компании сейчас смогут дать хорошую оплату, надо искать что-то онлайн, стабильное. Хотя какое оно стабильное, надо открывать свое дело в интернете, может делать магазин с доставкой? Да я так и не смогла этот английский выучить сколько уже ходила на курсы. Короче искать работу это мне вообще как настоящее испытание, ну и не думаю что без английского я кому-то нужна. Пока живем в бесплатном жилье, но тоже непонятно надолго это, вообще нужно ли возвращаться, в Киеве хотя бы есть дом, родные стены, ну и удобства все под рукой, но я боюсь ходить по улице вижу везде эти новости про растяжки, бомбы в молоке. Но и помощи от волонтеров больше есть и помощь с едой и вообще дома как то проще, здесь никого не знаю, но и знакомые если честно все уже разъехались и реально жизнь как с нуля. С мужем тоже сложно, постоянно приходится принимать решения, а он как будто оттягивает, но я же тоже не могу все время все делать. Короче мне реально не понятно как вообще быть, я боюсь что-то снова начинать, потому что не понятно на долго или скоро закончится.",cli
4,"Давайте выделим вопросы, которые я услышала и продолжим работу. Сейчас у вас нет основной работы для вас большой стресс искать новую, еще вы хотите вернуться в Киев, но не можете решить для себя стоит ли это сейчас делать, также вам хочется получить поддержку от мужа, но вы не можете получить той помощи на которую рассчитываете. Но самым большим испытанием для вас является то, что необходимо начать все с нуля. верно?",psy
5,"Да, но даже не сам факт начала. Понимаете я уже несколько раз все теряла и начинала ""жизнь с этого нуля"". Поэтому мне кажется тут у меня что-то вроде иммунитета. Страх не с самой необходимости, я просто реально понимаю что если я еще раз все потеряю мне может просто не хотеться что-то делать снова. Именно когда живешь в период от нуля до ""опять все хочется"" тут самое страшное. Ощущение, что теряешь время, которого и так постоянно мало, и параллельно же ничего не улучшается, тебе еще наваливается сверху, и нужно разгребать 

In [70]:
full_cli_text = ' '.join(i for i in cli_df.text.tolist()).strip()
full_psy_text = ' '.join(i for i in psy_df.text.tolist()).strip()
full_text = ' '.join(i for i in conversation_df.text.tolist()).strip()
full_text

'Добрый день, меня зовут Ирина. Ольга, у нас с вами на сегодня назначена консультация. Надеюсь, что наш разговор будет приятным и полезным. Расскажите с каким запросом вы пришли на нашу первую встречу? Добрый день, это моя первая консультация в жизни. Я не особо понимаю что именно нужно говорить. Но у меня накопилось достаточно много всего и мне просто хотелось поговорить о том, что сейчас происходит с моей жизнью, ну много чего, конечно, как и у всех, все таки война, переезды, куча всего короче. Просто хотелось чтобы кто-то услышал, помог мне немного разобраться в себе. Ольга, действительно сейчас многие проживают события к которым сложно быть готовым. Давайте начнем понемногу разбираться. Расскажите о себе. Где вы сейчас находитесь, в каких условиях живете, какие события происходят вокруг вас? Сейчас я чувствую себя относительно в безопасности. Мы переехали из Киева на западную Украину. Здесь можно сказать, что тихо, но я не особо понимаю что происходит. Мне словно приходится начинат

# Simple Solutions with Pretrained Transformers

In [71]:
def get_summary_phrases(context_list, func):
  sums = []
  for i in context_list:
    sums.append(func(i))
  return sums

## Bert2bert

bert2bert [model](https://huggingface.co/dmitry-vorobiev/rubert_ria_headlines ), initialized with the DeepPavlov/rubert-base-cased pretrained weights and fine-tuned on the first 99% of "Rossiya Segodnya" news dataset for 2 epochs.

In [72]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

MODEL_NAME = "dmitry-vorobiev/rubert_ria_headlines"

bert2bert_tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
bert2bert_model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

The following encoder weights were not tied to the decoder ['bert/pooler']
The following encoder weights were not tied to the decoder ['bert/pooler']


In [73]:
def extract_summary_bert2bert(text):
  encoded_batch = bert2bert_tokenizer.prepare_seq2seq_batch(
    [text],
    return_tensors="pt",
    padding="max_length",
    truncation=True,
    max_length=512)

  output_ids = bert2bert_model.generate(
    input_ids=encoded_batch["input_ids"],
    max_length=36,
    no_repeat_ngram_size=3,
    num_beams=5,
    top_k=0
  )

  headline = bert2bert_tokenizer.decode(output_ids[0], 
                            skip_special_tokens=True, 
                            clean_up_tokenization_spaces=False)
  return headline

In [74]:
bert2bert_cli_sums = get_summary_phrases(cli_df.text.tolist(), extract_summary_bert2bert)
bert2bert_psy_sums = get_summary_phrases(psy_df.text.tolist(), extract_summary_bert2bert)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3516: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


## T5 pretrained on Telegram dataset
Based on rut5-base [model](https://huggingface.co/IlyaGusev/rut5_base_headline_gen_telegram)

In [75]:
from transformers import AutoTokenizer, T5ForConditionalGeneration

model_name = "IlyaGusev/rut5_base_headline_gen_telegram"
t5_tokenizer = AutoTokenizer.from_pretrained(model_name)
t5_model = T5ForConditionalGeneration.from_pretrained(model_name)

In [76]:
def extract_summary_t5(article_text, max_length=600):
  input_ids = t5_tokenizer(
    [article_text],
    max_length=600,
    add_special_tokens=True,
    padding="max_length",
    truncation=True,
    return_tensors="pt"
  )["input_ids"]

  output_ids = t5_model.generate(
    input_ids=input_ids
  )[0]

  headline = t5_tokenizer.decode(output_ids, skip_special_tokens=True)
  return headline

In [77]:
t5_cli_sums = get_summary_phrases(cli_df.text.tolist(), extract_summary_t5)
t5_psy_sums = get_summary_phrases(psy_df.text.tolist(), extract_summary_t5)

## T5 pretrained on Gazeta dataset
[This](https://huggingface.co/IlyaGusev/rut5_base_sum_gazeta) is the model for abstractive summarization for Russian based on rut5-base.

In [78]:
model_name_t5_gazeta = "IlyaGusev/rut5_base_sum_gazeta"
tokenizer_t5_gazeta = AutoTokenizer.from_pretrained(model_name_t5_gazeta)
model_t5_gazeta = T5ForConditionalGeneration.from_pretrained(model_name_t5_gazeta)


In [79]:
def extract_summary_t5_gazeta(article_text):
  input_ids = tokenizer_t5_gazeta(
    [article_text],
    max_length=600,
    add_special_tokens=True,
    padding="max_length",
    truncation=True,
    return_tensors="pt"
  )["input_ids"]

  output_ids = model_t5_gazeta.generate(
    input_ids=input_ids,
    no_repeat_ngram_size=4
  )[0]

  summary = tokenizer_t5_gazeta.decode(output_ids, skip_special_tokens=True)
  return summary

In [80]:
t5_cli_sums_gazeta = get_summary_phrases(cli_df.text.tolist(), extract_summary_t5_gazeta)
t5_psy_sums_gazeta = get_summary_phrases(psy_df.text.tolist(), extract_summary_t5_gazeta)

# Analysing results

In [81]:
def print_n(list_n):
  for i in list_n:
    print(i)
  print()

Results with bert2bert 

In [82]:
print_n(bert2bert_cli_sums)
print_n(bert2bert_psy_sums)

татьяна сергеева
украина в безопасности, или как я люблю работу с англиискими деньгами
от нуля к блекаутым
сергеи
" я не хочу ехать в киев, но через неделю "
" я не хочу жить однои, а мы уже не будем жить "
" я хочу быть вместе с мужем "
" я уже не знаю, что я люблю, а я люблю себя? "
сергеи

сергеи
ольга, ольга
" я не знаю, что нужно начать все с нуля "
ответственность за принятые решения на себя?
что делать с мужем?
о том, что муж оттягивал принятие решении решении решении
" я не нужно никого ждать... "
как правильно договориться с семьеи. треилер фильма
" я хочу разобраться, что вы чувствуетеся в эмоциях "
сергеи сергеевич



Results with T5_telegram

In [83]:
print_n(t5_cli_sums)
print_n(t5_psy_sums)

Добрый день, моя первая консультация в жизни
Что делать, если наладится ничего не наладится, но ничего не наладится
Страх не с самой необходимости, я уже несколько раз все теряла и начинала "жизнь с этого нуля"
Не надо ждать коллективного решения, а сам тихо сделала и ни у кого не спрашиваешь
Что делать, если бы муж помог мне и согласился бы на переезд уже было бы проще
В постоянном ожидании чуда жить тоже не легко
Не хочу представлять, а если бы. Мне нравятся наши отношения, хотя такие штуки когда приходится ждать решения прям бесят
Я как будто живу с бомбой с часовым механизмом и даже не предлагаю никаких идей
Спасибо, реально интересно чего я чувствую иногда кажется что уже ничего

У нас с вами назначена консультация
Что происходит вокруг вас?
Что нужно знать о работе с нуля?
Вопрос: готовы ли брать ответственность за принятые решения на себя?
Вопрос о разводе с мужем: я так понимаю у вас разные точки зрения на какую то ситуацию с мужем?
Муж оттягивал принятие решений, когда муж отт

Results with T5_gazeta

In [84]:
print_n(t5_cli_sums_gazeta)
print_n(t5_psy_sums_gazeta)

Мне хотелось поговорить о том, что сейчас происходит с моей жизнью, а также о том как происходит сейчас. Я не особо понимаю, что именно нужно говорить. Просто хотелось чтобы кто-то услышал, помог мне разобраться в себе.
Я живу в Киеве и хочу найти работу в интернете, но не понимаю, почему я живу в этом городе, а не в стране, а в стране очень тихо и спокойно.
Я уже несколько раз все теряла и начинала жить в период от нуля до «опять все хочется». Мне кажется, что если я еще раз все потеряю, мне может просто не хотеться что-то делать снова.
Я пытаюсь балансировать между личной ответственностью за собственную жизнь и умением договориться. Не надо ждать коллективного решения, а сам тихо сделала и ни у кого не спрашивает.
Я не хочу ехать в Киев, я хочу чтобы мы вдвоем поехали, потому что завтра лучше не станет, а после завтра тоже тоже станет.
Я не хочу жить одна, а хочу понять, как можно было бы жить, если бы мы не были вместе, потому что я не хотела бы постоянно ждать, где бы я уже была. П

The performence of rut5-base trained on Telegram dataset is the best. Let's use it to show the main keypoints to the user

In [85]:
t5_sums = t5_cli_sums + t5_psy_sums
t5_sums_text = '. '.join(t5_sums)
t5_sums_text

'Добрый день, моя первая консультация в жизни. Что делать, если наладится ничего не наладится, но ничего не наладится. Страх не с самой необходимости, я уже несколько раз все теряла и начинала "жизнь с этого нуля". Не надо ждать коллективного решения, а сам тихо сделала и ни у кого не спрашиваешь. Что делать, если бы муж помог мне и согласился бы на переезд уже было бы проще. В постоянном ожидании чуда жить тоже не легко. Не хочу представлять, а если бы. Мне нравятся наши отношения, хотя такие штуки когда приходится ждать решения прям бесят. Я как будто живу с бомбой с часовым механизмом и даже не предлагаю никаких идей. Спасибо, реально интересно чего я чувствую иногда кажется что уже ничего. У нас с вами назначена консультация. Что происходит вокруг вас?. Что нужно знать о работе с нуля?. Вопрос: готовы ли брать ответственность за принятые решения на себя?. Вопрос о разводе с мужем: я так понимаю у вас разные точки зрения на какую то ситуацию с мужем?. Муж оттягивал принятие решений,

# Further Steps

## Vector Representation of Sentences with SentenceTransformer
[SentenceTransformers](https://www.sbert.net/examples/applications/computing-embeddings/README.html) is a Python framework for state-of-the-art sentence, text and image embeddings.
### Distiluse-base-multilingual-cased

This is a sentence-transformers [model](https://huggingface.co/sentence-transformers/distiluse-base-multilingual-cased): It maps sentences & paragraphs to a 512 dimensional dense vector space and can be used for tasks like clustering or semantic search.

Full Model Architecture:
```
SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: DistilBertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Dense({'in_features': 768, 'out_features': 512, 'bias': True, 'activation_function': 'torch.nn.modules.activation.Tanh'})
)
```



In [86]:
!pip install -U sentence-transformers

In [87]:
from sentence_transformers import SentenceTransformer, util, models

In [88]:
st_model = SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased')

In [89]:
sent_vect = lambda sentences: st_model.encode(sentences)

In [96]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize, sent_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [99]:
import nltk.data
s_tokenizer = nltk.data.load('tokenizers/punkt/russian.pickle')

In [100]:
def split_text(text):
  return [i for i in s_tokenizer.tokenize(text)]

In [101]:
split_t5_sums = split_text(t5_sums_text)

In [102]:
sentence_vectors = sent_vect(split_t5_sums)

In [103]:
sentence_vectors

array([[ 0.0054105 , -0.00886872,  0.00120969, ..., -0.01570556,
         0.00984144,  0.03023268],
       [ 0.00178432, -0.0087492 ,  0.03183902, ...,  0.00129839,
         0.01863876,  0.01968449],
       [ 0.00974022,  0.07614667,  0.03124227, ..., -0.06393679,
        -0.00111329, -0.01852025],
       ...,
       [-0.05058642, -0.00258173,  0.01237124, ..., -0.00330928,
        -0.01920856, -0.02056896],
       [-0.04808177,  0.05165537, -0.00559803, ...,  0.01668649,
        -0.01627786, -0.01267045],
       [ 0.02671837, -0.0282538 ,  0.01736572, ..., -0.03209884,
        -0.01977813, -0.05262212]], dtype=float32)

In [104]:
sentence_vectors.shape

(20, 512)

In [105]:
# similarity matrix
def form_sim_matr(sent_vectors):
  sim_mat = np.zeros([len(sent_vectors), len(sent_vectors)])
  for i in range(len(sent_vectors)):
    for j in range(len(sent_vectors)):
      if i != j:
        sim_mat[i][j] = cosine_similarity(sent_vectors[i].reshape(1,sent_vectors.shape[1]), sent_vectors[j].reshape(1,sent_vectors.shape[1]))[0,0]
  return sim_mat

In [106]:
all_sim_matr = form_sim_matr(sentence_vectors)

## Rank Sentences

In [115]:
def rank_sentences(text):
  vect = sent_vect(split_text(text))
  sim_mat = form_sim_matr(vect)
  nx_graph = nx.from_numpy_array(sim_mat)
  scores = nx.pagerank(nx_graph)
  ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(split_text(text))), reverse=True)
  return ranked_sentences

In [116]:
rank_sent = rank_sentences(t5_sums_text)

In [117]:
rank_sent

[(0.07315400099210567,
  'Вопрос о разводе с мужем: я так понимаю у вас разные точки зрения на какую то ситуацию с мужем?.'),
 (0.07044578567919878,
  'Что происходит с вами, когда вам не нужно никого ждать?.'),
 (0.06695937130263044,
  'Мне нравятся наши отношения, хотя такие штуки когда приходится ждать решения прям бесят.'),
 (0.0638885914329078,
  'Что делать, если бы муж помог мне и согласился бы на переезд уже было бы проще.'),
 (0.061058568421402885,
  'Спасибо, реально интересно чего я чувствую иногда кажется что уже ничего.'),
 (0.06090339630404084,
  'Муж оттягивал принятие решений, когда муж оттягивал принятие решений?.'),
 (0.05682808436738453,
  'Не надо ждать коллективного решения, а сам тихо сделала и ни у кого не спрашиваешь.'),
 (0.05501895219268985,
  'Что делать, если наладится ничего не наладится, но ничего не наладится.'),
 (0.05479266735012615,
  'Какие эмоции вы испытываете от "неопределенности" в решениях внутри семьи.'),
 (0.053513089752605346,
  'Я как будто ж

In [118]:
type(rank_sent)

list

In [120]:
rank_df = pd.DataFrame(rank_sent, columns=['rank', 'sent'])

In [121]:
rank_df

,rank,sent
0,0.073154,Вопрос о разводе с мужем: я так понимаю у вас разные точки зрения на какую то ситуацию с мужем?.
1,0.070446,"Что происходит с вами, когда вам не нужно никого ждать?."
2,0.066959,"Мне нравятся наши отношения, хотя такие штуки когда приходится ждать решения прям бесят."
3,0.063889,"Что делать, если бы муж помог мне и согласился бы на переезд уже было бы проще."
4,0.061059,"Спасибо, реально интересно чего я чувствую иногда кажется что уже ничего."
5,0.060903,"Муж оттягивал принятие решений, когда муж оттягивал принятие решений?."
6,0.056828,"Не надо ждать коллективного решения, а сам тихо сделала и ни у кого не спрашиваешь."
7,0.055019,"Что делать, если наладится ничего не наладится, но ничего не наладится."
8,0.054793,"Какие эмоции вы испытываете от ""неопределенности"" в решениях внутри семьи."
9,0.053513,Я как будто живу с бомбой с часовым механизмом и даже не предлагаю никаких идей.


# End-user Approach

```
summarize_text(text, num_bull_points=5, is_dialog = False)
```

The function summarizes key points from sentences with T5 Model, then builds vector representations of sentences, ranks them and chooses the top results.
The user can choose the number of final bullet points.

```
Parameters:
*num_bull_points* - the number of bullet points
*is_dialog* - tells whether there is a necessity to parse psychologist's and client's replies separately

Returns: list of tuples
```

## Summarizer

In [123]:
text_p = "Психолог: Добрый день, меня зовут Ирина. Ольга, у нас с вами на сегодня назначена консультация. Надеюсь, что наш разговор будет приятным и полезным. Расскажите с каким запросом вы пришли на нашу первую встречу? Ольга: Добрый день, это моя первая консультация в жизни. Я не особо понимаю что именно нужно говорить. Но у меня накопилось достаточно много всего и мне просто хотелось поговорить о том, что сейчас происходит с моей жизнью, ну много чего, конечно, как и у всех, все таки война, переезды, куча всего короче. Просто хотелось чтобы кто-то услышал, помог мне немного разобраться в себе. Психолог: Ольга, действительно сейчас многие проживают события к которым сложно быть готовым. Давайте начнем понемногу разбираться. Расскажите о себе. Где вы сейчас находитесь, в каких условиях живете, какие события происходят вокруг вас?"

In [124]:
def parse_dialog(text):
  test_list = text.split(' ')
  size = len(test_list)
  idx_list = [idx for idx, val in
            enumerate(test_list) if val.lower().startswith('ольга:') 
            or val.lower().startswith('о:') or val.lower().startswith('психолог:')]
  res = [test_list[i: j] for i, j in
        zip([0] + idx_list, idx_list + 
        ([size] if idx_list[-1] != size else []))]
  res.remove(res[0])
  for i in res:
    for j in i:
      if j.lower() in ['психолог:', 'о:', 'ольга:']:
        i.remove(j)
  res_list = [item for sublist in res for item in sublist]
  return ' '.join(res_list)


In [143]:
import random
def split_by_three(text):
  partitioned_t = []
  split_t = split_text(text)
  for i, j in enumerate(split_t):
    if i < len(split_t)-2 and i%3!=1 and i%3!=2:
      partitioned_t.append(split_t[i] + ' ' + split_t[i+1] + ' ' + split_t[i+2])
  partitioned_t.append(split_t[len(split_t)-2] + ' ' +split_t[len(split_t)-1])
  return partitioned_t

In [141]:
split_text(full_text)

['Добрый день, меня зовут Ирина.',
 'Ольга, у нас с вами на сегодня назначена консультация.',
 'Надеюсь, что наш разговор будет приятным и полезным.',
 'Расскажите с каким запросом вы пришли на нашу первую встречу?',
 'Добрый день, это моя первая консультация в жизни.',
 'Я не особо понимаю что именно нужно говорить.',
 'Но у меня накопилось достаточно много всего и мне просто хотелось поговорить о том, что сейчас происходит с моей жизнью, ну много чего, конечно, как и у всех, все таки война, переезды, куча всего короче.',
 'Просто хотелось чтобы кто-то услышал, помог мне немного разобраться в себе.',
 'Ольга, действительно сейчас многие проживают события к которым сложно быть готовым.',
 'Давайте начнем понемногу разбираться.',
 'Расскажите о себе.',
 'Где вы сейчас находитесь, в каких условиях живете, какие события происходят вокруг вас?',
 'Сейчас я чувствую себя относительно в безопасности.',
 'Мы переехали из Киева на западную Украину.',
 'Здесь можно сказать, что тихо, но я не ос

In [144]:
split_by_three(full_text)

['Добрый день, меня зовут Ирина. Ольга, у нас с вами на сегодня назначена консультация. Надеюсь, что наш разговор будет приятным и полезным.',
 'Расскажите с каким запросом вы пришли на нашу первую встречу? Добрый день, это моя первая консультация в жизни. Я не особо понимаю что именно нужно говорить.',
 'Но у меня накопилось достаточно много всего и мне просто хотелось поговорить о том, что сейчас происходит с моей жизнью, ну много чего, конечно, как и у всех, все таки война, переезды, куча всего короче. Просто хотелось чтобы кто-то услышал, помог мне немного разобраться в себе. Ольга, действительно сейчас многие проживают события к которым сложно быть готовым.',
 'Давайте начнем понемногу разбираться. Расскажите о себе. Где вы сейчас находитесь, в каких условиях живете, какие события происходят вокруг вас?',
 'Сейчас я чувствую себя относительно в безопасности. Мы переехали из Киева на западную Украину. Здесь можно сказать, что тихо, но я не особо понимаю что происходит.',
 'Мне слов

In [145]:
def summarize_text(text, num_bull_points=5, is_dialog = False):
  if is_dialog:
    text = parse_dialog(text)
  snts = '. '.join(get_summary_phrases(split_by_three(text), extract_summary_t5))
  rank_sent = rank_sentences(snts)[:num_bull_points]
  return rank_sent

In [146]:
summarize_text(full_psy_text)

[(0.15642583700955065, 'Что будет с первым заданием и что будет с ним?'),
 (0.1554790348348071,
  'Вопрос: готовы ли брать ответственность за принятые решения на себя?.'),
 (0.13514548371780047,
  'Что происходит с вами, когда вам не нужно никого ждать?.'),
 (0.11006592065710759,
  'Какие эмоции вы испытываете от "неопределенности" в решениях внутри семьи?.'),
 (0.10553255848836135, 'Что происходит вокруг вас?')]

In [155]:
summarize_text(full_cli_text, num_bull_points=3)

[(0.051268771624891706,
  'Не понятно как вообще быть, я боюсь что-то снова начинать.'),
 (0.048296365321179556,
  'Не хочу жить одна, мне нравится наши отношения если бы не это.'),
 (0.04806796000001405,
  'Блин я уже иногда думаю, а если бы мы не были вместе, где бы я уже была?.')]

In [147]:
 test1 = summarize_text(full_text, num_bull_points=10)
 test1

[(0.03917125065460286,
  'Не хочу жить одна, мне нравится наши отношения если бы не это.'),
 (0.038109096807927706,
  'Потерять финансовую стабильность мне реально тяжело, ждать что что-то наладится, но ничего не наладится.'),
 (0.03762186009764585,
  'Страх не с самой необходимости, я уже несколько раз все теряла и начинала "жизнь с этого нуля".'),
 (0.03728173800122908,
  'Я как будто живу с бомбой с часовым механизмом и даже не предлагаю никаких идей.'),
 (0.03724113833164983, 'Не понимаю что такое жить с партнером.'),
 (0.03697364314167993,
  'Блин я уже иногда думаю, а если бы мы не были вместе, где бы я уже была?.'),
 (0.0368034845137374,
  'Когда живешь в период от нуля до "опять все хочется" это самое страшное.'),
 (0.03651416879283714,
  'Муж постоянно откладывает принятие решений, а я потом что-то решу.'),
 (0.03599832541237095,
  'Если бы я осталась одна мне было бы намного проще, но я хочу быть вместе с мужем, оберегать наше партнерство.'),
 (0.035798416847785536, 'Мне слов

In [153]:
def print_n(list_n):
  for i, j in list_n:
    print(j)
  print()

In [154]:
print_n(test1)

Не хочу жить одна, мне нравится наши отношения если бы не это.
Потерять финансовую стабильность мне реально тяжело, ждать что что-то наладится, но ничего не наладится.
Страх не с самой необходимости, я уже несколько раз все теряла и начинала "жизнь с этого нуля".
Я как будто живу с бомбой с часовым механизмом и даже не предлагаю никаких идей.
Не понимаю что такое жить с партнером.
Блин я уже иногда думаю, а если бы мы не были вместе, где бы я уже была?.
Когда живешь в период от нуля до "опять все хочется" это самое страшное.
Муж постоянно откладывает принятие решений, а я потом что-то решу.
Если бы я осталась одна мне было бы намного проще, но я хочу быть вместе с мужем, оберегать наше партнерство.
Мне словно приходится начинать все с нуля.



# Evaluation

## Load evaluation dataset

In [156]:
eval_all = pd.read_csv('https://raw.githubusercontent.com/Teasotea/textSummarization/main/data/evaluation_summary_all.txt', delimiter = "\t", header=None).set_axis(['summaries'], axis=1, inplace=False)
eval_cli = pd.read_csv('https://raw.githubusercontent.com/Teasotea/textSummarization/main/data/evaluation_summary_cli.txt', delimiter = "\t", header=None).set_axis(['summaries'], axis=1, inplace=False)
eval_psy = pd.read_csv('https://raw.githubusercontent.com/Teasotea/textSummarization/main/data/evaluation_summary_psy.txt', delimiter = "\t", header=None).set_axis(['summaries'], axis=1, inplace=False)


In [158]:
psy_sum = summarize_text(full_psy_text, num_bull_points=9)

## ROUGE: Evaluation​​ [Metric](https://ilmoirfan.com/rouge-an-evaluation-metric-for-text-summarization/)​​ for Text Summarization Task

### ROUGE score (Recall-Oriented Understudy for Gisting Evaluation)

The algorithm to compute ROUGE score considers consecutive tokens a.k.a. n-grams. The n-grams from one text (e.g. the human-written summary) are compared to the n-grams of the other text (e.g. the machine-written summary). **A large overlap of n-grams results in a high ROUGE score** and a low overlap — in a low ROUGE score.

When the aim is abstractive summarization ROUGE is best used **only as an initial indicator** of how much the machine-written summary overlaps with the human written summary, because it does not take into account the semantic meaning and the factual accuracy of the summaries.

Pros and cons of using [ROUGE](https://towardsdatascience.com/to-rouge-or-not-to-rouge-6a5f3552ea45)

In [160]:
!pip install Rouge

In [161]:
from rouge import Rouge 
rouge = Rouge()

In [162]:
serie_to_text = lambda s: '. '.join(s.tolist())

In [164]:
summary = ' '.join([j for i,j in psy_sum])
reference = serie_to_text(eval_psy.summaries)

In [165]:
summary # summary made by model

'Что будет с первым заданием и что будет с ним? Вопрос: готовы ли брать ответственность за принятые решения на себя?. Что происходит с вами, когда вам не нужно никого ждать?. Какие эмоции вы испытываете от "неопределенности" в решениях внутри семьи?. Что происходит вокруг вас? У нас с вами на сегодня назначена консультация. Ответы на вопросы, которые я услышала и продолжим работу. Домашнее задание для мужа. Как начать все с нуля, чтобы начать все с нуля.'

In [166]:
reference # summary made by human

'Цель встречи. Расскажите о себе. Нет основной работы. Желание вернуться в Киев. Нет поддержки от мужа. Страх начать с нуля. Страх неопределенности. Нужно разобратся с емоциями. ДЗ: Записывать в дневник ситуацию и эмоции'

In [167]:
rouge.get_scores('Цель встречи', 'Цель встречи !')

[{'rouge-1': {'f': 0.7999999952000001, 'p': 1.0, 'r': 0.6666666666666666},
  'rouge-2': {'f': 0.6666666622222223, 'p': 1.0, 'r': 0.5},
  'rouge-l': {'f': 0.7999999952000001, 'p': 1.0, 'r': 0.6666666666666666}}]

In [168]:
rouge.get_scores(summary, reference)

[{'rouge-1': {'f': 0.17777777340987666,
   'p': 0.13114754098360656,
   'r': 0.27586206896551724},
  'rouge-2': {'f': 0.019607838831219707,
   'p': 0.014285714285714285,
   'r': 0.03125},
  'rouge-l': {'f': 0.17777777340987666,
   'p': 0.13114754098360656,
   'r': 0.27586206896551724}}]

In [169]:
rouge.get_scores(summary, full_psy_text)

[{'rouge-1': {'f': 0.35125447687003,
   'p': 0.8032786885245902,
   'r': 0.22477064220183487},
  'rouge-2': {'f': 0.21680216494737847,
   'p': 0.5714285714285714,
   'r': 0.13377926421404682},
  'rouge-l': {'f': 0.35125447687003,
   'p': 0.8032786885245902,
   'r': 0.22477064220183487}}]

# Some ideas for approach improvements



*   Explore dataset (EDA): do linguistical and statistical analysis, for example, word frequency calculation, normalization, considering the priority of words in a language. Fine research on that topic is in this [article](https://www.ijert.org/research/text-summarizer-using-abstractive-and-extractive-method-IJERTV3IS050821.pdf )
*   Develop text summarization solution using RNNs and LSTM (the core idea explained [here](https://www.analyticsvidhya.com/blog/2019/06/comprehensive-guide-text-summarization-using-deep-learning-python/ )) and compare the results
*   Improve Text Preprocessing: do the plural resolution, abbreviation resolution, synonym resolution, and remove personal information, like names, cities, and so on with the help of the NER model
*   Fine-Tune Transformers on the custom dataset, which consists of psychological sessions recordings